# Initialising LUSID

In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd

import helper_functions as fun 

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('LUSID API Version: ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.5.3337.0


![Scopes](img/paper-lusid.gif)

## 1) Creating a logically separated environment for each analyst

In [2]:
# Fetch our scopes
analyst_scope_code = fun.create_analyst_scope()


Analyst Scope Code: analyst-paper-37c7-6237-a461-e4


*Our LUSID environment now looks like the below. We will update this with each step.*

![Scopes](img/paper-scopes.gif)

## 2) Creating our tradeable instrument universe for each analyst

In [3]:
# Import our instrument universe from a CSV file using Pandas
instrument_universe = pd.read_csv('data/paper-instruments.csv')
# Look at the first 10 instruments
instrument_universe.sort_values(by="instrument_name").head(n=10)

,instrument_name,currency,figi,ticker,isin,sedol,client_internal
0,AVEVA GRP,GBP,BBG000C21Y87,AVV,GB00BBG9VN75,BBG9VN7,imd_56344535
1,BAE SYS.,GBP,BBG000BD5TW4,BA.,GB0002634946,263494,imd_56344983
2,BATM ADVANCED,GBP,BBG000BFJD77,BVC,IL0010849045,911146,imd_56344098
3,BTG,GBP,BBG000BZZK79,BTG,GB0001001592,100159,imd_56344345
4,COBHAM,GBP,BBG000BS6810,COB,GB00B07KD360,B07KD36,imd_56344363
5,COMPUTACENTER,GBP,BBG000BN7CL9,CCC,GB00BV9FP302,BV9FP30,imd_56344296
6,CONSORT MED.,GBP,BBG000BD8278,CSRT,GB0000946276,94627,imd_56344187
7,DIALIGHT,GBP,BBG000BLQNG4,DIA,GB0033057794,3305779,imd_56344572
8,GENUS,GBP,BBG000CTMKX7,GNS,GB0002074580,207458,imd_56344789
9,GRESHAM TECH,GBP,BBG000BS2592,GHT,GB0008808825,880882,imd_56344132


In [4]:
batch_upsert_request = fun.batch_upsert(instrument_universe, 
                 client)   

Instruments Successfully Upserted: 


In [5]:
# Call LUSID to upsert our batch
instrument_response = client.instruments.upsert_instruments(
    instruments=batch_upsert_request)

# Pretty print the response from LUSID
prettyprint.instrument_response(instrument_response, identifier='Figi').sort_values(by="Instrument").head(10)

Instruments Successfully Upserted: 


,Instrument,Figi ID,LUSID Instrument ID
11,AVEVA GRP,BBG000C21Y87,LUID_C63M3W9L
28,BAE SYS.,BBG000BD5TW4,LUID_PZY35CYB
14,BATM ADVANCED,BBG000BFJD77,LUID_RN4DVOPG
5,BTG,BBG000BZZK79,LUID_9G72A4LZ
17,COBHAM,BBG000BS6810,LUID_4S3J1BCC
21,COMPUTACENTER,BBG000BN7CL9,LUID_0ZN4YO4U
15,CONSORT MED.,BBG000BD8278,LUID_JCNUA8A4
26,DIALIGHT,BBG000BLQNG4,LUID_3G6G53QI
7,GENUS,BBG000CTMKX7,LUID_NM4VWVZG
1,GRESHAM TECH,BBG000BS2592,LUID_TCDOZL9W


Our LUSID environment now has an instrument master containing all of our instruments that we will be trading.

![Scopes](img/paper-instrumentmaster.gif)

## 3) Creating a transactions portfolio

A transactions portfolio is initalised to hold all transactions made by an analyst. 

A second portfolio is also initalised which will store the index of the instrument universe to be used as a benchmark.

In [6]:
# Define unique codes for each of our portfolios
transaction_portfolio_code = 'LSE_FTSE_techMarkFocus_transactions_DEMO'
reference_portfolio_code = 'LSE_FTSE_techMarkFocus_reference_DEMO'
# Pretty print our codes
prettyprint.heading('Transaction Portfolio Code', transaction_portfolio_code)
prettyprint.heading('Reference Portfolio Code', reference_portfolio_code)

# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3))

descriptions = "Paper transaction portfolio_DEMO"
portfolio_codes = [transaction_portfolio_code, reference_portfolio_code]


fun.request_transaction_portfolio_creation(transaction_portfolio_code, 
                                           portfolio_creation_date, 
                                           analyst_scope_code, 
                                           descriptions,
                                           client)

portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3))

fun.request_reference_portfolio_creation(reference_portfolio_code, 
                                         portfolio_creation_date, 
                                         analyst_scope_code, 
                                         client)

Transaction Portfolio Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Reference Portfolio Code: LSE_FTSE_techMarkFocus_reference_DEMO
Portfolio Created
Scope: analyst-paper-37c7-6237-a461-e4
Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Portfolio Effective From: 2019-09-29 16:47:18.636765+00:00
Portfolio Created On: 2019-10-02 16:47:18.987260+00:00

Portfolio Created
Scope: analyst-paper-37c7-6237-a461-e4
Code: LSE_FTSE_techMarkFocus_reference_DEMO
Portfolio Effective From: 2019-09-29 16:47:19.075077+00:00
Portfolio Created On: 2019-10-02 16:47:19.424187+00:00



![Scopes](img/paper-referenceportfolio.gif)

## 4) Populating our analyst's portfolio with a starting cash balance

In [7]:
# Set the date from which the cash balance will apply to be just after portfolio creation
holdings_effective_date = (datetime.now(pytz.UTC) - timedelta(days=2.9))
# Define our initial cash balance
initial_cash_balance = 30000000

# # Create a holding adjustment to set our initial cash balance
fun.populate_with_cash(holdings_effective_date, 
                       initial_cash_balance, 
                       analyst_scope_code, 
                       transaction_portfolio_code, 
                       client)


Holdings Successfully Set for Portfolio
Scope: analyst-paper-37c7-6237-a461-e4
Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Holdings Effective From: 2019-09-29 19:11:19.546257+00:00
Holdings Created On: 2019-10-02 16:47:19.911980+00:00



Our transactions portfolio is now populated with an initial cash balance and is no longer empty.

![Scopes](img/paper-transactionsportfoliofilled.gif)

## 5) Populating our reference portfolio 

The initaised reference portfolio is populated with correct weights such that it tracks the market as closely as possible.

In [8]:
# Import the market capitalisation of each of our instruments in the index
instrument_market_cap = pd.read_csv('data/paper-weights.csv')
# Take a look at the first 10 market capitalisations
instrument_market_cap.sort_values(by="instrument_name").head(n=10)

,instrument_name,currency,figi,ticker,isin,sedol,marketcap
0,AVEVA GRP,GBP,BBG000C21Y87,AVV,GB00BBG9VN75,BBG9VN7,4502.68
1,BAE SYS.,GBP,BBG000BD5TW4,BA.,GB0002634946,0263494,16349.66
2,BATM ADVANCED,GBP,BBG000BFJD77,BVC,IL0010849045,911146,185.25
3,BTG,GBP,BBG000BZZK79,BTG,GB0001001592,100159,3202.20
4,COBHAM,GBP,BBG000BS6810,COB,GB00B07KD360,B07KD36,2579.90
5,COMPUTACENTER,GBP,BBG000BN7CL9,CCC,GB00BV9FP302,BV9FP30,1161.96
6,CONSORT MED.,GBP,BBG000BD8278,CSRT,GB0000946276,0094627,422.56
7,DIALIGHT,GBP,BBG000BLQNG4,DIA,GB0033057794,3305779,133.39
8,GENUS,GBP,BBG000CTMKX7,GNS,GB0002074580,0207458,1455.72
9,GRESHAM TECH,GBP,BBG000BS2592,GHT,GB0008808825,0880882,64.34


In [9]:
# upsert contituents to LUSID
fun.upsert_constituents(instrument_market_cap, 
                        holdings_effective_date, 
                        analyst_scope_code, 
                        reference_portfolio_code, 
                        client)

Constituents Upserted


![Scopes](img/paper-referenceportfoliofilled.gif)

## 6) Allow our analysts to trade across their tradeable instrument universe and add their transactions to their transaction portfolio

In [10]:
# Import our analyst's trades from a CSV file
analyst_transactions = import_data.fetch_client_transactions(
    'paper-transactions.csv',
    days_back=2)
analyst_transactions.sort_values(by="transaction_id").head(n=10)


,transaction_id,type,portfolio,instrument_name,instrument_uid,transaction_date,settlement_date,units,transaction_price,transaction_currency,total_cost,strategy,description
0,tid_124398219481,Buy,LSE_FTSE_techMarkFocus_transactions,AVEVA GRP,BBG000C21Y87,2019-10-01T11:37:41.760239+00:00,2019-10-03T11:37:41.760239+00:00,265600,28.94,GBP,7686464.00,quantitativeSignal,Purchase
4,tid_223239424244,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2019-10-01T10:22:42.386034+00:00,2019-10-03T10:22:42.386034+00:00,750000,0.00,GBP,69.45,fundamentalAnalysis,Brokerage Fees
9,tid_239394829484,Buy,LSE_FTSE_techMarkFocus_transactions,DIALIGHT,BBG000BLQNG4,2019-09-30T10:38:55.975166+00:00,2019-10-02T10:38:55.975166+00:00,1500000,4.18,GBP,6270000.00,quantitativeSignal,Purchase
17,tid_324599699055,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2019-10-01T09:34:07.904455+00:00,2019-10-03T09:34:07.904455+00:00,234000,0.00,GBP,458.64,quantitativeSignal,Stamp Duty
15,tid_334250655566,Buy,LSE_FTSE_techMarkFocus_transactions,MICROGEN,BBG000BDSS88,2019-09-30T14:27:13.854446+00:00,2019-10-02T14:27:13.854446+00:00,234000,3.92,GBP,917280.00,quantitativeSignal,Purchase
1,tid_339423984894,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2019-09-30T10:42:34.446313+00:00,2019-10-02T10:42:34.446313+00:00,265600,0.01,GBP,1537.29,quantitativeSignal,Brokerage Fees
2,tid_339423984896,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2019-10-01T16:08:47.320873+00:00,2019-10-03T16:08:47.320873+00:00,265600,0.01,GBP,3843.23,quantitativeSignal,Stamp Duty
14,tid_342398598555,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2019-09-30T11:29:12.978759+00:00,2019-10-02T11:29:12.978759+00:00,345000,0.01,GBP,3795.00,technicalAnalysis,Stamp Duty
6,tid_342398989895,SellShort,LSE_FTSE_techMarkFocus_transactions,COMPUTACENTER,BBG000BN7CL9,2019-09-30T12:17:55.880230+00:00,2019-10-02T12:17:55.880230+00:00,150000,10.50,GBP,1575000.00,fundamentalAnalysis,Purchase
20,tid_342453598349,FundsOut,LSE_FTSE_techMarkFocus_transactions,GBP_Cash,GBP,2019-10-01T14:17:52.073066+00:00,2019-10-03T14:17:52.073066+00:00,543000,0.01,GBP,3692.40,technicalAnalysis,Stamp Duty


Custom properties can be defined in LUSID. In this example, a trading strategy is specified created as a **property**, this allows grouping of transactions for the purpose of measuring performance, valuation etc...

In [11]:
# Create a request to define our strategy property
strategy_property_key = fun.request_define_property(domain = 'Transaction', 
                                                    scope = analyst_scope_code, 
                                                    code = 'strategy', 
                                                    display_name='strategy', 
                                                    client=client)

Strategy Property Key: : Transaction/analyst-paper-37c7-6237-a461-e4/strategy


In [12]:
# upsert all analyist transactions to LUSID
fun.upsert_trades(analyst_transactions, 
                  strategy_property_key, 
                  analyst_scope_code, 
                  transaction_portfolio_code, 
                  client)

Transactions Successfully Upserted into Portfolio
Scope: analyst-paper-37c7-6237-a461-e4
Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Transactions Effective From: 2019-10-01 16:37:15.721324+00:00
Transactions Created On: 2019-10-02 16:47:21.780356+00:00



Our transaction portfolio is now populated with our analyst's paper trades.

## 7) Create an analytics store to hold the latest prices of our tradeable instrument universe

In [13]:
# Import our instrument prices from a CSV file
instrument_prices = pd.read_csv('data/paper-analytics.csv')
# Pretty print our pricing
instrument_prices.sort_values(by="instrument_name").head(n=10)

,instrument_name,currency,figi,price_original,price_current,ticker,isin,sedol
0,AVEVA GRP,GBP,BBG000C21Y87,27.84,30.32,AVV,GB00BBG9VN75,BBG9VN7
1,BAE SYS.,GBP,BBG000BD5TW4,5.15,4.78,BA.,GB0002634946,263494
2,BATM ADVANCED,GBP,BBG000BFJD77,0.46,0.42,BVC,IL0010849045,911146
3,BTG,GBP,BBG000BZZK79,8.28,8.97,BTG,GB0001001592,100159
4,COBHAM,GBP,BBG000BS6810,1.07,1.02,COB,GB00B07KD360,B07KD36
5,COMPUTACENTER,GBP,BBG000BN7CL9,10.68,11.46,CCC,GB00BV9FP302,BV9FP30
6,CONSORT MED.,GBP,BBG000BD8278,8.70,9.31,CSRT,GB0000946276,94627
7,DIALIGHT,GBP,BBG000BLQNG4,3.92,3.55,DIA,GB0033057794,3305779
8,GENUS,GBP,BBG000CTMKX7,22.46,20.60,GNS,GB0002074580,207458
9,GRESHAM TECH,GBP,BBG000BS2592,0.94,1.00,GHT,GB0008808825,880882


In [14]:
# Set our analytics effective dates
analytics_effective_date = datetime.now(pytz.UTC) - timedelta(days=3)
today = datetime.now(pytz.UTC)
analytics_store_dates = [analytics_effective_date, today]

fun.create_instrument_analytics(analytics_effective_date, 
                                today, 
                                analytics_store_dates, 
                                instrument_prices, 
                                analyst_scope_code, 
                                client)
    

Analytics Set


![Scopes](img/paper-analyticsstore.gif)

## 8) Value each portfolio using our analytics store

In [15]:
# create aggregation request
aggregation_request = fun.create_aggregation_request(analyst_scope_code, today, transaction_portfolio_code, client)

# Call LUSID to aggregate across all of our portfolios
aggregated_portfolio = client.aggregation.get_aggregation_by_portfolio(
    scope=analyst_scope_code,
    code=transaction_portfolio_code,
    request=aggregation_request)

aggregated_portfolio_df = pd.DataFrame(aggregated_portfolio.data)
aggregated_portfolio_df.head(n=10)

,Holding/default/SubHoldingKey,Sum(Holding/default/Units),Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Price)
0,Currency=GBP,4364776.00,4364776.00,4364776.00,19.00
1,LusidInstrumentId=LUID_3G6G53QI/GBP,1500000.00,6270000.00,5327280.00,3.55
2,LusidInstrumentId=LUID_05MDQMS5/GBP,120000.00,1582800.00,1603546.80,13.36
3,LusidInstrumentId=LUID_0ZN4YO4U/GBP,-150000.00,-1575000.00,-1718946.00,11.46
4,LusidInstrumentId=LUID_WJNQRY3P/GBP,98000.00,2156000.00,1912019.20,19.51
5,LusidInstrumentId=LUID_QQ3OG8O2/GBP,234000.00,917280.00,843094.98,3.60
6,LusidInstrumentId=LUID_NM4VWVZG/GBP,345000.00,7590000.00,7105557.90,20.60
7,LusidInstrumentId=LUID_XZ8VE9ED/GBP,55000.00,305250.00,308154.00,5.60
8,LusidInstrumentId=LUID_C63M3W9L/GBP,265600.00,7686464.00,8052397.06,30.32
9,LusidInstrumentId=LUID_RN4DVOPG/GBP,750000.00,347250.00,318431.25,0.42


## 9) Compare performance across the analyst's reference and transaction portfolios

In [16]:
# create index portfolio
index_portfolio_code = 'LSE_FTSE_techMarkFocus_index_DEMO'
prettyprint.heading('Index Portfolio Code', index_portfolio_code)

description = 'Index transaction portfolio_DEMO'
fun.request_transaction_portfolio_creation(index_portfolio_code, 
                                           portfolio_creation_date, 
                                           analyst_scope_code, 
                                           description,
                                           client)


Index Portfolio Code: LSE_FTSE_techMarkFocus_index_DEMO
Portfolio Created
Scope: analyst-paper-37c7-6237-a461-e4
Code: LSE_FTSE_techMarkFocus_index_DEMO
Portfolio Effective From: 2019-09-29 16:47:19.075077+00:00
Portfolio Created On: 2019-10-02 16:47:29.579671+00:00



We now have an index portfolio that we can use to hold our index and obtain our benchmark.

![Scopes](img/paper-indexportfolio.gif)

In [17]:
# setup an index portfolio
index_setup = fun.setup_index(analyst_scope_code, reference_portfolio_code, instrument_prices, client)

# Call LUSID to set all of our holdings to the initial index level
adjust_holdings_response = client.transaction_portfolios.set_holdings(
    scope=analyst_scope_code,
    code=index_portfolio_code,
    effective_at=portfolio_creation_date,
    holding_adjustments=index_setup)

prettyprint.set_holdings_response(
    adjust_holdings_response, 
    analyst_scope_code, 
    index_portfolio_code)

Holdings Successfully Set for Portfolio
Scope: analyst-paper-37c7-6237-a461-e4
Code: LSE_FTSE_techMarkFocus_index_DEMO
Holdings Effective From: 2019-09-29 16:47:19.075077+00:00
Holdings Created On: 2019-10-02 16:47:32.789036+00:00



We now have an index portfolio that we can value.

![Scopes](img/paper-indexportfoliofilled.gif)

In [18]:
fun.create_analytics_aggregation_request(analyst_scope_code, index_portfolio_code, today, client)


Initial Index Level :999.98
Current Index Level :995.97
Return :-0.4013%
